In [8]:
!pip install yt-dlp


   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   --- ------------------------------------ 0.3/3.2 MB ? eta -:--:--
   ------------- -------------------------- 1.0/3.2 MB 3.3 MB/s eta 0:00:01
   ---------------------- ----------------- 1.8/3.2 MB 3.4 MB/s eta 0:00:01
   -------------------------------- ------- 2.6/3.2 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 3.4 MB/s eta 0:00:00


In [15]:
import yt_dlp

channel_url = 'https://youtube.com/playlist?list=PLRnlRGar-_296KTsVL0R6MEbpwJzD8ppA&si=awSWL7yNSOPmkiYp'  # playlist de las conferencias mañaneras

ydl_opts = {
    'quiet': True,
    'extract_flat': True,
    'force_generic_extractor': True,
}

video_urls = []

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(channel_url, download=False)
    entries = info.get('entries', [])
    
    for entry in entries:
        if entry.get('id'):
            video_url = f"https://www.youtube.com/watch?v={entry['id']}"
            video_urls.append(video_url)

print(f"Total videos encontrados: {len(video_urls)}")




Total videos encontrados: 1389


In [17]:
from pytube import YouTube
import pandas as pd
import re
import time

# Lista de URLs de los videos (la puedes llenar manualmente o desde otro script)
video_urls = [
    # Aquí agregas tus URLs de video, como ejemplo:
    "https://www.youtube.com/live/Q5Z01pPwoEM?si=Qvvy6BNmX6HL4KVI",
    # Agrega más URLs...
]

# Función para extraer subtítulos
def extract_captions(url):
    try:
        yt = YouTube(url)
        title = yt.title
        caption = yt.captions.get_by_language_code('es')  # Extraemos los subtítulos en español
        if caption:
            srt = caption.generate_srt_captions()
            # Limpiar los timestamps
            clean_text = re.sub(r"\d+\n\d\d:\d\d:\d\d,\d+ --> \d\d:\d\d:\d\d,\d+\n", "", srt)
            clean_text = clean_text.strip()
            return {'Título': title, 'URL': url, 'Texto': clean_text}
        else:
            print(f"⚠️ No hay subtítulos en español para: {title}")
            return None
    except Exception as e:
        print(f"❌ Error al procesar {url}: {e}")
        return None

# Procesar todos los videos
def process_videos(video_urls):
    data = []
    for idx, url in enumerate(video_urls):
        result = extract_captions(url)
        if result:
            data.append(result)
        time.sleep(1)  # Evitar sobrecargar YouTube
    return data

# Guardar los resultados en CSV
def save_to_csv(data, filename="subtitulos_extraidos.csv"):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"✅ Subtítulos guardados en '{filename}'")

# Ejecutar el proceso
if __name__ == "__main__":
    print("📝 Comenzando la extracción de subtítulos...")
    data = process_videos(video_urls)
    
    if data:
        print("💾 Guardando los resultados...")
        save_to_csv(data)
    else:
        print("❌ No se extrajeron subtítulos.")



📝 Comenzando la extracción de subtítulos...
❌ Error al procesar https://www.youtube.com/live/Q5Z01pPwoEM?si=Qvvy6BNmX6HL4KVI: HTTP Error 400: Bad Request
❌ No se extrajeron subtítulos.


In [1]:
!pip install pytube